In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import os, sys, pathlib, random, time, pickle, copy, json
from tqdm import tqdm

/home/tsuman/All_Files/Program_Files/miniconda3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:1")
# device = torch.device("cpu")

In [3]:
# SEED = 147
# SEED = 258
SEED = 369

torch.manual_seed(SEED)
np.random.seed(SEED)

In [4]:
import torch.optim as optim
from torch.utils import data

In [5]:
# cifar_train = transforms.Compose([
#     transforms.RandomCrop(size=32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
#         std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
#     ),
# ])

# cifar_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
#         std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
#     ),
# ])

# train_dataset = datasets.CIFAR10(root="../../../../../_Datasets/cifar10/", train=True, download=True, transform=cifar_train)
# test_dataset = datasets.CIFAR10(root="../../../../../_Datasets/cifar10/", train=False, download=True, transform=cifar_test)

In [6]:
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=2)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [7]:
cifar_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4865, 0.4409],
        std=[0.2009, 0.1984, 0.2023],
    ),
])

cifar_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4865, 0.4409],
        std=[0.2009, 0.1984, 0.2023],
    ),
])

train_dataset = datasets.CIFAR100(root="../../../../../_Datasets/cifar100/", train=True, download=True, transform=cifar_train)
test_dataset = datasets.CIFAR100(root="../../../../../_Datasets/cifar100/", train=False, download=True, transform=cifar_test)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Model

In [9]:
class MlpBLock(nn.Module):
    
    def __init__(self, input_dim, hidden_layers_ratio=[2], actf=nn.GELU):
        super().__init__()
        self.input_dim = input_dim
        #### convert hidden layers ratio to list if integer is inputted
        if isinstance(hidden_layers_ratio, int):
            hidden_layers_ratio = [hidden_layers_ratio]
            
        self.hlr = [1]+hidden_layers_ratio+[1]
        
        self.mlp = []
        ### for 1 hidden layer, we iterate 2 times
        for h in range(len(self.hlr)-1):
            i, o = int(self.hlr[h]*self.input_dim),\
                    int(self.hlr[h+1]*self.input_dim)
            self.mlp.append(nn.Linear(i, o))
            self.mlp.append(actf())
        self.mlp = self.mlp[:-1]
        
        self.mlp = nn.Sequential(*self.mlp)
        
    def forward(self, x):
        return self.mlp(x)

In [10]:
MlpBLock(2, [3,4])

MlpBLock(
  (mlp): Sequential(
    (0): Linear(in_features=2, out_features=6, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=6, out_features=8, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=8, out_features=2, bias=True)
  )
)

## MLP-Mixer 

In [11]:
class MixerBlock(nn.Module):
    
    def __init__(self, patch_dim, channel_dim):
        super().__init__()
        
        self.ln0 = nn.LayerNorm(channel_dim)
        self.mlp_patch = MlpBLock(patch_dim, [2])
        self.ln1 = nn.LayerNorm(channel_dim)
        self.mlp_channel = MlpBLock(channel_dim, [2])
    
    def forward(self, x):
        ## x has shape-> N, nP, nC/hidden_dims; C=Channel, P=Patch
        
        ######## !!!! Can use same mixer on shape of -> N, C, P;
        
        #### mix per patch
        y = self.ln0(x) ### per channel layer normalization ?? 
        y = torch.swapaxes(y, -1, -2)
        y = self.mlp_patch(y)
        y = torch.swapaxes(y, -1, -2)
        x = x+y
        
        #### mix per channel 
        y = self.ln1(x)
        y = self.mlp_channel(y)
        x = x+y
        return x

In [12]:
class MlpMixer(nn.Module):
    
    def __init__(self, image_dim:tuple, patch_size:tuple, hidden_expansion:float, num_blocks:int, num_classes:int):
        super().__init__()
        
        self.img_dim = image_dim ### must contain (C, H, W) or (H, W)
        self.scaler = nn.UpsamplingBilinear2d(size=(self.img_dim[-2], self.img_dim[-1]))
        
        ### find patch dim
        d0 = int(image_dim[-2]/patch_size[0])
        d1 = int(image_dim[-1]/patch_size[1])
        assert d0*patch_size[0]==image_dim[-2], "Image must be divisible into patch size"
        assert d1*patch_size[1]==image_dim[-1], "Image must be divisible into patch size"
#         self.d0, self.d1 = d0, d1 ### number of patches in each axis
        __patch_size = patch_size[0]*patch_size[1]*image_dim[0] ## number of channels in each patch
    
        ### find channel dim
        channel_size = d0*d1 ## number of patches
        
        ### after the number of channels are changed
        init_dim = __patch_size
#         final_dim = int(patch_size[0]*patch_size[1]*hidden_expansion)
        final_dim = int(init_dim*hidden_expansion)

        self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)
        #### rescale the patches (patch wise image non preserving transform, unlike bilinear interpolation)
        self.channel_change = nn.Linear(init_dim, final_dim)
        print(f"MLP Mixer : Channes per patch -> Initial:{init_dim} Final:{final_dim}")
        
        
        self.channel_dim = final_dim
        self.patch_dim = channel_size
        
        self.mixer_blocks = []
        for i in range(num_blocks):
            self.mixer_blocks.append(MixerBlock(self.patch_dim, self.channel_dim))
        self.mixer_blocks = nn.Sequential(*self.mixer_blocks)
        
        self.linear = nn.Linear(self.patch_dim*self.channel_dim, num_classes)
        
        
    def forward(self, x):
        bs = x.shape[0]
        x = self.scaler(x)
        x = self.unfold(x).swapaxes(-1, -2)
        x = self.channel_change(x)
        x = self.mixer_blocks(x)
        x = self.linear(x.view(bs, -1))
        return x

In [13]:
# class MlpMixer(nn.Module):
    
#     def __init__(self, input_channels:int, hidden_image_dim:tuple, patch_size:tuple, num_blocks:int, num_classes:int, use1x1scale=True):
#         super().__init__()
        
#         self.img_dim = hidden_image_dim ### must contain (C, H, W) or (H, W)
        
#         ### find patch dim
#         d0 = int(self.img_dim[-2]/patch_size[0])
#         d1 = int(self.img_dim[-1]/patch_size[1])
#         assert d0*patch_size[0]==self.img_dim[-2], "Image must be divisible into patch size"
#         assert d1*patch_size[1]==self.img_dim[-1], "Image must be divisible into patch size"
        
#         ### find channel dim
#         channel_size = d0*d1 ## number of patches
        
#         ### after the number of channels are changed
#         init_dim = patch_size[0]*patch_size[1]*input_channels ## number of channels in each patch
#         final_dim = patch_size[0]*patch_size[1]*self.img_dim[0]

#         self.scaler = nn.UpsamplingBilinear2d(size=(self.img_dim[-2], self.img_dim[-1]))
        
#         if use1x1scale:
#             self.conv1x1 = nn.Conv2d(input_channels, self.img_dim[0], kernel_size=1, stride=1)
#             if input_channels == self.img_dim[0]:
#                 self.conv1x1 = nn.Identity()
#         else:
#             #### rescale the patches (patch wise image non preserving transform, unlike bilinear interpolation)
#             self.channel_change = nn.Linear(init_dim, final_dim) ## apply after unfold
            
#         self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)

#         print(f"MLP Mixer : Channes per patch -> Initial:{init_dim} Final:{final_dim}")
        
#         ### after the axis are swapped
#         self.channel_dim = final_dim
#         self.patch_dim = channel_size
        
#         self.mixer_blocks = []
#         for i in range(num_blocks):
#             self.mixer_blocks.append(MixerBlock(self.patch_dim, self.channel_dim))
#         self.mixer_blocks = nn.Sequential(*self.mixer_blocks)
        
#         self.linear = nn.Linear(self.patch_dim*self.channel_dim, num_classes)
        
        
#     def forward(self, x):
#         bs = x.shape[0]
#         x = self.scaler(x) ## scale to high dimension
#         x = self.conv1x1(x)
#         x = self.unfold(x).swapaxes(-1, -2).contiguous() ## convert to patches, bring channels first
# #         x = self.channel_change(x) ## change the number of channels
#         x = self.mixer_blocks(x) ## the mixer architecture
#         x = self.linear(x.view(bs, -1)) ## classify
#         return x

In [14]:
mixer = MlpMixer((3, 32, 32), (4, 4), hidden_expansion=2.5, num_blocks=1, num_classes=10)
mixer

MLP Mixer : Channes per patch -> Initial:48 Final:120


MlpMixer(
  (scaler): UpsamplingBilinear2d(size=(32, 32), mode=bilinear)
  (unfold): Unfold(kernel_size=(4, 4), dilation=1, padding=0, stride=(4, 4))
  (channel_change): Linear(in_features=48, out_features=120, bias=True)
  (mixer_blocks): Sequential(
    (0): MixerBlock(
      (ln0): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=64, out_features=128, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=128, out_features=64, bias=True)
        )
      )
      (ln1): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
      (mlp_channel): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=120, out_features=240, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=240, out_features=120, bias=True)
        )
      )
    )
  )
  (linear): Linear(in_features=7680, out_features=10, bias=True)
)

In [15]:
print("number of params: ", sum(p.numel() for p in mixer.parameters())) 

number of params:  157706


In [16]:
mixer(torch.randn(1, 3, 32, 32))

tensor([[-0.1648, -0.0510, -0.0486, -0.1505, -0.3137, -0.5226,  0.0023,  0.5233,
         -0.2587, -0.5398]], grad_fn=<AddmmBackward0>)

## Patch Mixer

In [17]:
class PatchMixerBlock(nn.Module):
    
    def __init__(self, patch_size, num_channel):
        super().__init__()
        self.patch_size = patch_size
        
#         self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)
        ps = None
        if isinstance(patch_size, int):
            ps = patch_size**2
        else:
            ps = patch_size[0]*patch_size[1]
        ps = ps*num_channel
        
        self.ln0 = nn.LayerNorm(ps)
        self.mlp_patch = MlpBLock(ps, [2])
        
#         self.fold = nn.Fold(kernel_size=patch_size, stride=patch_size)
    
    def forward(self, x):
        ## x has shape-> N, C, H, W; C=Channel
        
        sz = x.shape
        
        y = nn.functional.unfold(x, 
                                 kernel_size=self.patch_size, 
                                 stride=self.patch_size
                                )
        #### mix per patch
        y = torch.swapaxes(y, -1, -2)
        y = self.ln0(y) 
        y = self.mlp_patch(y)
        y = torch.swapaxes(y, -1, -2)
        
        y = nn.functional.fold(y, (sz[-2], sz[-1]), 
                               kernel_size=self.patch_size, 
                               stride=self.patch_size
                              )
        x = x+y
        return x

In [18]:
pmb = PatchMixerBlock(8, 3)
pmb

PatchMixerBlock(
  (ln0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (mlp_patch): MlpBLock(
    (mlp): Sequential(
      (0): Linear(in_features=192, out_features=384, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=384, out_features=192, bias=True)
    )
  )
)

In [19]:
# pmb(torch.randn(1, 3, 35, 35)).shape

In [20]:
def get_factors(n):
    facts = []
    for i in range(2, n+1):
        if n%i == 0:
            facts.append(i)
    return facts

class PatchMlpMixer(nn.Module):
    
    def __init__(self, image_dim:tuple, patch_sizes:tuple, hidden_channels:int, num_blocks:int, num_classes:int):
        super().__init__()
        
        self.img_dim = image_dim ### must contain (C, H, W)
        self.target_dim = np.prod(patch_sizes)
        
        ### find number of channel for input, the channel is 
        num_channel = image_dim[0]
        
        self.conv1x1 = nn.Conv2d(num_channel, hidden_channels, kernel_size=1, stride=1)
        if num_channel == hidden_channels:
            self.conv1x1 = nn.Identity()
        
        self.mixer_blocks = []
        for i in range(num_blocks):
            for ps in patch_sizes:
                self.mixer_blocks.append(PatchMixerBlock(ps, hidden_channels))
                
        self.mixer_blocks = nn.Sequential(*self.mixer_blocks)
        self.linear = nn.Linear(self.target_dim*self.target_dim*hidden_channels, num_classes)
    
    def forward(self, x):
        bs = x.shape[0]
        
        x = nn.functional.interpolate(x, size=self.target_dim, mode='bilinear', align_corners=True)
        
        x = self.conv1x1(x) 
        x = self.mixer_blocks(x)
        x = self.linear(x.view(bs, -1))
        return x

In [21]:
4*3*5

60

In [22]:
patch_mixer = PatchMlpMixer((3, 35, 35), patch_sizes=[5, 7], hidden_channels=3, num_blocks=1, num_classes=10)

In [23]:
patch_mixer

PatchMlpMixer(
  (conv1x1): Identity()
  (mixer_blocks): Sequential(
    (0): PatchMixerBlock(
      (ln0): LayerNorm((75,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=75, out_features=150, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=150, out_features=75, bias=True)
        )
      )
    )
    (1): PatchMixerBlock(
      (ln0): LayerNorm((147,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=147, out_features=294, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=294, out_features=147, bias=True)
        )
      )
    )
  )
  (linear): Linear(in_features=3675, out_features=10, bias=True)
)

In [24]:
print("number of params: ", sum(p.numel() for p in patch_mixer.parameters())) 

number of params:  146806


In [25]:
patch_mixer(torch.randn(1, 3, 32, 32)).shape

torch.Size([1, 10])

#### Final Model

In [26]:
### Model 1 : balanced with (1) of PatchOnly ; higher image scaling, low channel scaling for similar params
#### it has large dimension of final hidden unit.
## 81, 150
# model = MlpMixer((3, 5*9, 5*9), (5, 5), hidden_expansion=2, num_blocks=10, num_classes=10)
### balanced model with 4*9 expansion as well
## 81, 144
model = MlpMixer((3, 4*9, 4*9), (4, 4), hidden_expansion=3.0, num_blocks=10, num_classes=10)

### Model 2 : use simple method of expanding only hidden dimension/channel
## 64, 135
# model = MlpMixer((3, 32, 32), (4, 4), hidden_expansion=3.2, num_blocks=10, num_classes=10)


model = model.to(device)

MLP Mixer : Channes per patch -> Initial:48 Final:144


In [27]:
model

MlpMixer(
  (scaler): UpsamplingBilinear2d(size=(36, 36), mode=bilinear)
  (unfold): Unfold(kernel_size=(4, 4), dilation=1, padding=0, stride=(4, 4))
  (channel_change): Linear(in_features=48, out_features=144, bias=True)
  (mixer_blocks): Sequential(
    (0): MixerBlock(
      (ln0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=81, out_features=162, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=162, out_features=81, bias=True)
        )
      )
      (ln1): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
      (mlp_channel): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=144, out_features=288, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=288, out_features=144, bias=True)
        )
      )
    )
    (1): MixerBlock(
      (ln0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)

In [28]:
# print("number of params: ", sum(p.numel() for p in model.parameters())) 
print("number of params: ", sum(p.numel() for p in model.mixer_blocks.parameters())) 

number of params:  1104390


In [29]:
model(torch.randn(1, 3, 32, 32).to(device)).shape

torch.Size([1, 10])

In [30]:
### Model 1 : balanced using best settings for default patch mixer without scaling hidden channels.
model = PatchMlpMixer((3, 35, 35), patch_sizes=[5,7], hidden_channels=3, num_blocks=10, num_classes=10)

model = model.to(device)

In [31]:
model

PatchMlpMixer(
  (conv1x1): Identity()
  (mixer_blocks): Sequential(
    (0): PatchMixerBlock(
      (ln0): LayerNorm((75,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=75, out_features=150, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=150, out_features=75, bias=True)
        )
      )
    )
    (1): PatchMixerBlock(
      (ln0): LayerNorm((147,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=147, out_features=294, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=294, out_features=147, bias=True)
        )
      )
    )
    (2): PatchMixerBlock(
      (ln0): LayerNorm((75,), eps=1e-05, elementwise_affine=True)
      (mlp_patch): MlpBLock(
        (mlp): Sequential(
          (0): Linear(in_features=75, out_features=150, bias=True)
          (1): GELU(a

In [32]:
print("number of params: ", sum(p.numel() for p in model.mixer_blocks.parameters())) 

number of params:  1100460


In [33]:
model(torch.randn(1, 3, 32, 32).to(device)).shape

torch.Size([1, 10])

In [34]:
print("number of params: ", sum(p.numel() for p in model.parameters())) 
## Patch ||  1137220
## Mixer ||  1141703

number of params:  1137220


## Training

In [35]:
# model_name = f'mlp_mixer_c10_s{SEED}'
# model_name = f'temp_c10_s{SEED}'

In [36]:
# EPOCHS = 200
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [37]:
# STAT ={'train_stat':[], 'test_stat':[]}

In [38]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    STAT['train_stat'].append((epoch, train_loss/(batch_idx+1), 100.*correct/total)) ### (Epochs, Loss, Acc)
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [39]:
best_acc = -1
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    time_taken = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)

            start = time.time()

            outputs = model(inputs)

            start = time.time()-start
            time_taken.append(start)

            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    STAT['test_stat'].append((epoch, test_loss/(batch_idx+1), 100.*correct/total, np.mean(time_taken))) ### (Epochs, Loss, Acc, time)
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}.pth')
        best_acc = acc
        
    with open(f"./output/{model_name}_data.json", 'w') as f:
        json.dump(STAT, f, indent=0)

In [40]:
# start_epoch = 0  # start from epoch 0 or last checkpoint epoch
# resume = False

# if resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('./models'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load(f'./models/{model_name}.pth')
#     model.load_state_dict(checkpoint['model'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

In [41]:
# ### Train the whole damn thing

# for epoch in range(start_epoch, start_epoch+EPOCHS): ## for 200 epochs
#     train(epoch)
#     test(epoch)
#     scheduler.step()

In [42]:
# best_acc

In [43]:
# checkpoint = torch.load(f'./models/{model_name}.pth')
# best_acc = checkpoint['acc']
# start_epoch = checkpoint['epoch']

# best_acc, start_epoch

In [44]:
# model.load_state_dict(checkpoint['model'])

In [45]:
# model

In [46]:
# STAT

In [47]:
# train_stat = np.array(STAT['train_stat'])
# test_stat = np.array(STAT['test_stat'])

In [48]:
# plt.plot(train_stat[:,1], label='train')
# plt.plot(test_stat[:,1], label='test')
# plt.ylabel("Loss")
# plt.legend()
# plt.savefig(f"./output/plots/{model_name}_loss.svg")
# plt.show()

In [49]:
# plt.plot(train_stat[:,2], label='train')
# plt.plot(test_stat[:,2], label='test')
# plt.ylabel("Accuracy")
# plt.legend()
# plt.savefig(f"./output/plots/{model_name}_accs.svg")
# plt.show()

## Benchmark Training

In [50]:
def get_data_loaders(seed, ds):
    BS = 64
    if ds == 'c100': BS = 128
    torch.manual_seed(seed)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BS, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BS, shuffle=False, num_workers=2)
    return train_loader, test_loader

In [51]:
def benchmark():
    global model, optimizer, train_loader, test_loader, model_name, criterion, STAT, best_acc
    EPOCHS = 200
    criterion = nn.CrossEntropyLoss()
    lr = 0.001
    DS = 'c100'
#     DS = 'c10'
    for SEED in [369]:
        for num_layers in [7, 10]:

            for i in range(3): ## 3 models training
                print("Experiment index:", i)
                train_loader, test_loader = get_data_loaders(SEED, DS)
                num_cls = 10
                if DS=='c100': num_cls = 100
                ### FOR ORIGINAL MIXER V0
                if i==0:
                    ## hard core ignore
#                     if SEED==258 and num_layers==7: continue
                    model = MlpMixer((3, 4*9, 4*9), (4, 4), hidden_expansion=3.0, num_blocks=num_layers, num_classes=num_cls)
                    model_name = f'original_mixer0_l{num_layers}_{DS}_s{SEED}'
                elif i == 1:
                    ### FOR ORIGINAL MIXER V1
#                     if SEED==258 and num_layers==7: continue
                    model = MlpMixer((3, 32, 32), (4, 4), hidden_expansion=3.2, num_blocks=num_layers, num_classes=num_cls)
                    model_name = f'original_mixer1_l{num_layers}_{DS}_s{SEED}'
                elif i == 2:
#                     if SEED==369 and num_layers==7: continue
                    model = PatchMlpMixer((3, 35, 35), patch_sizes=[5,7], hidden_channels=3, num_blocks=num_layers, num_classes=num_cls)
                    model_name = f'patchonly_mixer0_l{num_layers}_{DS}_s{SEED}'
                else:
                    print("JPT........!!!!")
                model = model.to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

                num_params = sum(p.numel() for p in model.parameters())

                model_name = "03.0_" + model_name
                print(f"EXPERIMENTING FOR : {model_name} | params: {num_params}  .......\n.......")
                
#                 continue
                STAT ={'train_stat':[], 'test_stat':[], 'num_params':num_params}
                best_acc = -1
                for epoch in range(0, EPOCHS): ## for 200 epochs
                    train(epoch)
                    test(epoch)
                    scheduler.step()
                print(f"Training finished\n")
                pass
            pass
        pass
    return 0           

In [ ]:
benchmark()

Experiment index: 0
MLP Mixer : Channes per patch -> Initial:48 Final:144
EXPERIMENTING FOR : 03.0_original_mixer0_l7_c100_s369 | params: 1946629  .......
.......


100%|███████████████████████████████████████████████████| 391/391 [00:12<00:00, 31.19it/s]


[Train] 0 Loss: 3.658 | Acc: 16.148 8074/50000


100%|█████████████████████████████████████████████████████| 79/79 [00:01<00:00, 65.64it/s]


[Test] 0 Loss: 3.103 | Acc: 25.030 2503/10000
Saving..


...

100%|███████████████████████████████████████████████████| 391/391 [00:27<00:00, 14.42it/s]


[Train] 19 Loss: 0.858 | Acc: 74.690 37345/50000


100%|█████████████████████████████████████████████████████| 79/79 [00:02<00:00, 38.39it/s]


[Test] 19 Loss: 2.284 | Acc: 52.840 5284/10000


 19%|█████████▉                                          | 75/391 [00:05<00:20, 15.78it/s]

In [ ]:
asdfasdfwdf

# Models -- New Test 
1. Scaling is different for original mlp mixer so not useful for benchmarking
2. MLP mixer uses channel wise mixing, here channels are number of patches; it changes differntly from patch size when scaling.

In [ ]:
class MlpBLock(nn.Module):
    
    def __init__(self, input_dim, hidden_layers_ratio=[2], actf=nn.GELU):
        super().__init__()
        self.input_dim = input_dim
        #### convert hidden layers ratio to list if integer is inputted
        if isinstance(hidden_layers_ratio, int):
            hidden_layers_ratio = [hidden_layers_ratio]
            
        self.hlr = [1]+hidden_layers_ratio+[1]
        
        self.mlp = []
        ### for 1 hidden layer, we iterate 2 times
        for h in range(len(self.hlr)-1):
            i, o = int(self.hlr[h]*self.input_dim),\
                    int(self.hlr[h+1]*self.input_dim)
            self.mlp.append(nn.Linear(i, o))
            self.mlp.append(actf())
        self.mlp = self.mlp[:-1]
        
        self.mlp = nn.Sequential(*self.mlp)
        
    def forward(self, x):
        return self.mlp(x)

In [ ]:
MlpBLock(2, [3,4])

## MLP-Mixer 

In [ ]:
class MixerBlock(nn.Module):
    
    def __init__(self, patch_dim, channel_dim):
        super().__init__()
        
        self.ln0 = nn.LayerNorm(channel_dim)
        self.mlp_patch = MlpBLock(patch_dim, [2])
        self.ln1 = nn.LayerNorm(channel_dim)
        self.mlp_channel = MlpBLock(channel_dim, [2])
    
    def forward(self, x):
        ## x has shape-> N, nP, nC/hidden_dims; C=Channel, P=Patch
        
        ######## !!!! Can use same mixer on shape of -> N, C, P;
        
        #### mix per patch
        y = self.ln0(x) ### per channel layer normalization ?? 
        y = torch.swapaxes(y, -1, -2)
        y = self.mlp_patch(y)
        y = torch.swapaxes(y, -1, -2)
        x = x+y
        
        #### mix per channel 
        y = self.ln1(x)
        y = self.mlp_channel(y)
        x = x+y
        return x

In [ ]:
class MlpMixer(nn.Module):
    
    def __init__(self, input_channels:int, hidden_image_dim:tuple, patch_size:tuple, num_blocks:int, num_classes:int, use1x1scale=True):
        super().__init__()
        
        self.img_dim = hidden_image_dim ### must contain (C, H, W) or (H, W)
        
        ### find patch dim
        d0 = int(self.img_dim[-2]/patch_size[0])
        d1 = int(self.img_dim[-1]/patch_size[1])
        assert d0*patch_size[0]==self.img_dim[-2], "Image must be divisible into patch size"
        assert d1*patch_size[1]==self.img_dim[-1], "Image must be divisible into patch size"
        
        ### find channel dim
        channel_size = d0*d1 ## number of patches
        
        ### after the number of channels are changed
        init_dim = patch_size[0]*patch_size[1]*input_channels ## number of channels in each patch
        final_dim = patch_size[0]*patch_size[1]*self.img_dim[0]

        self.scaler = nn.UpsamplingBilinear2d(size=(self.img_dim[-2], self.img_dim[-1]))
        
        if use1x1scale:
            self.conv1x1 = nn.Conv2d(input_channels, self.img_dim[0], kernel_size=1, stride=1)
            if input_channels == self.img_dim[0]:
                self.conv1x1 = nn.Identity()
        else:
            #### rescale the patches (patch wise image non preserving transform, unlike bilinear interpolation)
            self.channel_change = nn.Linear(init_dim, final_dim) ## apply after unfold
            
        self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)

        print(f"MLP Mixer : Channes per patch -> Initial:{init_dim} Final:{final_dim}")
        
        ### after the axis are swapped
        self.channel_dim = final_dim
        self.patch_dim = channel_size
        
        self.mixer_blocks = []
        for i in range(num_blocks):
            self.mixer_blocks.append(MixerBlock(self.patch_dim, self.channel_dim))
        self.mixer_blocks = nn.Sequential(*self.mixer_blocks)
        
        self.linear = nn.Linear(self.patch_dim*self.channel_dim, num_classes)
        
        
    def forward(self, x):
        bs = x.shape[0]
        x = self.scaler(x) ## scale to high dimension
        x = self.conv1x1(x)
        x = self.unfold(x).swapaxes(-1, -2).contiguous() ## convert to patches, bring channels first
#         x = self.channel_change(x) ## change the number of channels
        x = self.mixer_blocks(x) ## the mixer architecture
        x = self.linear(x.view(bs, -1)) ## classify
        return x

In [ ]:
mixer = MlpMixer(3, (5, 35, 35), (5, 5), num_blocks=1, num_classes=10, use1x1scale=False)
mixer

In [ ]:
mixer(torch.randn(1, 3, 32, 32))

## Patch Mixer

In [ ]:
class PatchMixerBlock(nn.Module):
    
    def __init__(self, patch_size, num_channel):
        super().__init__()
        self.patch_size = patch_size
        
#         self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)
        ps = None
        if isinstance(patch_size, int):
            ps = patch_size**2
        else:
            ps = patch_size[0]*patch_size[1]
        ps = ps*num_channel
        
        self.ln0 = nn.LayerNorm(ps)
        self.mlp_patch = MlpBLock(ps, [2])
        
#         self.fold = nn.Fold(kernel_size=patch_size, stride=patch_size)
    
    def forward(self, x):
        ## x has shape-> N, C, H, W; C=Channel
        
        sz = x.shape
        
        y = nn.functional.unfold(x, 
                                 kernel_size=self.patch_size, 
                                 stride=self.patch_size
                                )
        #### mix per patch
        y = torch.swapaxes(y, -1, -2)
        y = self.ln0(y) 
        y = self.mlp_patch(y)
        y = torch.swapaxes(y, -1, -2)
        
        y = nn.functional.fold(y, (sz[-2], sz[-1]), 
                               kernel_size=self.patch_size, 
                               stride=self.patch_size
                              )
        x = x+y
        return x

In [ ]:
pmb = PatchMixerBlock(7, 5)
pmb

In [ ]:
pmb(torch.randn(1, 5, 35, 35)).shape

In [ ]:
def get_factors(n):
    facts = []
    for i in range(2, n+1):
        if n%i == 0:
            facts.append(i)
    return facts

class PatchMlpMixer(nn.Module):
    
    def __init__(self, input_channels:int, hidden_image_dim:tuple, patch_sizes:list, num_blocks:int, num_classes:int):
        super().__init__()
        
        self.img_dim = hidden_image_dim ### must contain (C, H, W)
        self.scaler = nn.UpsamplingBilinear2d(size=(self.img_dim[-2], self.img_dim[-1]))
        
        assert np.prod(patch_sizes) == hidden_image_dim[-1], "The product of patches must equal image dimension"
        assert np.prod(patch_sizes) == hidden_image_dim[-2], "The product of patches must equal image dimension"
        
        ### find number of channel for input, the channel is 
        self.conv1x1 = nn.Conv2d(input_channels, self.img_dim[0], kernel_size=1, stride=1)
        if input_channels == self.img_dim[0]:
            self.conv1x1 = nn.Identity()
        
        self.mixer_blocks = []
        for i in range(num_blocks):
            for ps in patch_sizes:
                self.mixer_blocks.append(PatchMixerBlock(ps, self.img_dim[0]))
                
        self.mixer_blocks = nn.Sequential(*self.mixer_blocks)
        self.linear = nn.Linear(np.prod(self.img_dim), num_classes)
    
    def forward(self, x):
        bs = x.shape[0]
        x = self.scaler(x)
        x = self.conv1x1(x)
        x = self.mixer_blocks(x)
        x = self.linear(x.view(bs, -1))
        return x

In [ ]:
patch_mixer = PatchMlpMixer(3, (5, 35, 35), patch_sizes=[5, 7], num_blocks=1, num_classes=10)

In [ ]:
patch_mixer

In [ ]:
patch_mixer(torch.randn(1, 3, 32, 32)).shape

#### Final Model

In [ ]:
model = MlpMixer(3, (9, 32, 32), patch_size=(4, 4), num_blocks=10, num_classes=10)
model = model.to(device)

In [ ]:
print("number of params: ", sum(p.numel() for p in model.parameters())) 

In [ ]:
model

In [ ]:
model = PatchMlpMixer(3, (3, 35, 35), patch_sizes=[5, 7], num_blocks=10, num_classes=10)
model = model.to(device)

In [ ]:
print("number of params: ", sum(p.numel() for p in model.parameters())) 

In [ ]:
model

In [ ]:
print("number of params: ", sum(p.numel() for p in model.parameters())) 
## Patch ||  1137220
## Mixer ||  1141703